# 헤더파일 선언

계산에 필요한 모든 함수들을 미리 모듈화해서 메인 코드의 가독성을 올린다.  

추후 수정 및 추가에도 용이하다.  

In [ ]:
"""

#ifndef FUNCTIONS_H
#define FUNCTIONS_H

void zeros(double** array, int rows, int cols);
double randn();
void randnArray(double** array, int rows, int cols);
void randArray(double** array, int rows, int cols);
void divide_matrix(double** array, int rows, int cols, double value);
void mul_matrix(double** array1, double** array2, int rows, int cols, double value);
void add_matrix(double** array, int rows, int cols, double value);
void dot_product(double** array, double** array1, double** array2, int a, int b, int c);
void sigmoid(double** array1, double** array2, int row, int col);
void relu(double** array1, double** array2, int row, int col);
void softmax(double** array1, double** array2, int row, int col);
void log_matrix(double** array1, double** array2, int row, int col);
void elementwise_mul(double** array, double** array1, double** array2, int row, int col);
void elementwise_minus(double** array, double** array1, double** array2, int row, int col);
void elementwise_add(double** array, double** array1, double** array2, int row, int col);
void matrix_minus_from_one(double** array1, double** array2, int row, int col);
double sum(double** array, int row, int col);
void matrix_transpose(double** result, double** matrix, int row, int col);
void relu_back(double** result_array, double** array, int row, int col);
int find_max_index(double** array, int col);
void printArray(double** array, int rows, int cols);

#endif

"""

# 함수 설명  

모든 계산 함수는 호출 전 계산 결과 size만큼 동적 할당을 한 다음에 연산 결과를 받아옴  

1. zeros  
행렬을 생성할 때 행렬의 내부 값들을 모두 0으로 한다.  

2. randn  
정규 분포 난수를 생성하는 함수이다. 가중치를 초기화할 떄 사용한다.  

3. randnArray  
위의 randn 함수를 호출해 정규 분포 난수로 이루어진 행렬을 반환  

4. divide_matrix  
행렬을 요소 별로 주어진 값으로 나눔  

5. mul_matrix  
행렬을 요소 별로 주어진 값으로 곱함  

6. add_matrix  
행렬을 요소 별로 주어진 값으로 더함  

7. dot_product  
a x b * b x c 행렬이 들어오면  내적 연산을 해 => a x c 사이즈의 행렬을 반환  

8. sigmoid  
output layer에서 시그모이드 활성화 함수 적용  

9. relu
hidden layer에서 relu 활성화 함수 적용  

10. softmax  
output layer에서 softmax 활성화 함수 적용
class 많을 때는 sigmoid보다 성능이 좋다.  
softmax를 사용할 것임  

11. log_matrix  
행렬의 각 요소에 log를 취해줌  

12. elementwise_mul   
두 행렬을 받으면 행렬의 요소끼리 곱해서 반환해 줌  

13. elementwise_minus  
두 행렬을 받으면 행렬의 앞 행렬의 값에서 뒤 행렬의 값을 빼서 반환해 줌    

14. elementwise_add  
두 행렬을 받으면 행렬의 요소끼리 더해서 반환해 줌  

15. matrix_minus_from_one  
행렬의 요소를 1에서 빼는 함수 cross entropy loss 계산 시 사용  

16. sum  
행렬을 받으면 요소끼리 전부 더 해서 반환해 줌  

17. matrix_transpose  
행렬 전치 함수 (행과 열을 바꿈)  
전치된 행렬의 행과 열 수는 원본 행렬의 열과 행 수와 같음  

18. relu_back  
역전파 단계에서 relu의 편미분 값을 구함  

19. find_max_index  
one hot encoding된 class에서 가장 큰 값이 예측한 정답값임  
가장 큰 값이 무엇인지 반환함  

20. printArray  
테스트용으로 행렬을 출력하기 위한 함수  


In [ ]:
"""
#include<stdio.h>
#include <stdlib.h>
#include <string.h>

#define _USE_MATH_DEFINES   
#include <math.h>

#include "functions.h"

모든 요소를 0으로 초기화
void zeros(double** array, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            array[i][j] = 0;
        }
    }
}

정규 분포 난수 생성 함수
double randn() {
    double u1 = (double)rand() / RAND_MAX; // 0에서 1 사이의 난수 생성
    double u2 = (double)rand() / RAND_MAX; // 0에서 1 사이의 또 다른 난수 생성
    double z = sqrt(-2 * log(u1)) * cos(2 * M_PI * u2); // 정규 분포로 변환

    return z;
}

정규 분포 난수 행렬 생성 함수
void randnArray(double** array, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            array[i][j] = randn(); // 정규 분포 난수로 초기화
        }
    }
}

0에서 1 사이의 난수 행렬 생성 함수
void randArray(double** array, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            array[i][j] = (double)rand() / RAND_MAX;
        }
    }
}

행렬을 주어진 값으로 나누는 함수
void divide_matrix(double** array, int rows, int cols, double value) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            array[i][j] = array[i][j] / value;
        }
    }
}

행렬을 주어진 값으로 곱하는 함수
void mul_matrix(double** array1, double** array2, int rows, int cols, double value) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            array1[i][j] = array2[i][j] * value;
        }
    }
}

행렬을 주어진 값으로 더하는 함수
void add_matrix(double** array, int rows, int cols, double value) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            array[i][j] = array[i][j] + value;
        }
    }
}

a x b * b x c 행렬이 들어오면  내적 연산을 해 => a x c 사이즈의 행렬을 반환
void dot_product(double** array, double** array1, double** array2, int a, int b, int c) {

    for (int i = 0; i < a; i++) {
        for (int j = 0; j < c; j++) {
            double product = 0;
            for (int k = 0; k < b; k++) {
                product += array1[i][k] * array2[k][j];
            }
            array[i][j] = product;
        }
    }
}

sigmoid 함수
void sigmoid(double** array1, double** array2, int row, int col) {
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            array1[i][j] = 1 / (1 + exp(-1 * array2[i][j]));
        }
    }
}

relu 함수
void relu(double** array1, double** array2, int row, int col) {
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            if (array2[i][j] > 0)  array1[i][j] = array2[i][j];
            else array1[i][j] = 0;
        }
    }
}

softmax 함수
void softmax(double** array1, double** array2, int row, int col) {

    double max_value = array2[0][find_max_index(array2, col)];
    double sum = 0;
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            sum += exp(array2[i][j] - max_value);
        }
    }
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            array1[i][j] = exp(array2[i][j] - max_value) / sum;
        }
    }
}


행렬의 각 요소에 log 취하는 함수
void log_matrix(double** array1, double** array2, int row, int col) {
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            array1[i][j] = log(array2[i][j]);
        }
    }
}

행렬의 요소끼리 곱하는 함수
void elementwise_mul(double** array, double** array1, double** array2, int row, int col) {
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            array[i][j] = array1[i][j] * array2[i][j];
        }
    }
}

행렬의 요소끼리 더하는 함수
void elementwise_add(double** array, double** array1, double** array2, int row, int col) {

    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            array[i][j] = array1[i][j] + array2[i][j];
        }
    }
}

행렬의 요소끼리 빼는 함수
void elementwise_minus(double** array, double** array1, double** array2, int row, int col) {

    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            array[i][j] = array1[i][j] - array2[i][j];
        }
    }
}

행렬의 요소를 1에서 빼는 함수
void matrix_minus_from_one(double** array1, double** array2, int row, int col) {
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            array1[i][j] = 1 - array2[i][j];
        }
    }
}

행렬의 모든 요소의 합을 구하는 함수
double sum(double** array, int row, int col) {

    double result = 0;
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            result += array[i][j];
        }
    }
    free(array);
    return result;
}

행렬 전치 함수 (행과 열을 바꿈)
전치된 행렬의 행과 열 수는 원본 행렬의 열과 행 수와 같음
void matrix_transpose(double** result, double** matrix, int row, int col) {
    
    for (int i = 0; i < col; i++) {
        for (int j = 0; j < row; j++) {
            result[i][j] = matrix[j][i];
        }
    }
}

행렬의 열에서 가장 큰 값의 인덱스를 구하는 함수
int find_max_index(double** array, int col) {
    int max_index = 0;
    double max_value = array[0][0];

    for (int i = 1; i < col; i++) {
        if (array[0][i] > max_value) {
            max_index = i;
            max_value = array[0][i];
        }
    }
    return max_index;
}

relu 함수의 역전파 함수
void relu_back(double** result_array, double** array, int row, int col) {
    for (int i = 0; i < row; i++) {
        for (int j = 0; j < col; j++) {
            if (array[i][j] > 0)  result_array[i][j] = 1;
            else result_array[i][j] = 0;
        }
    }
}

배열 출력 함수 (테스트용)
void printArray(double** array, int rows, int cols) {
    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            printf("%lf ", array[i][j]);
        }
        printf("\n");
    }
    printf("\n");
}
"""

# 헤더파일 호출 및 전역변수(Hyper parameter, 신경망 내부 변수) 선언

In [ ]:
"""
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include "functions.h" 

double** w2;
double** b2;
double** w3;
double** b3;
double** w4;
double** b4;
double** a1;
double** z1;
double** a2;
double** z2;
double** a3;
double** z3;
double** a4;
double** z4;


hyper parameter 설정
int input_nodes = 256; // 16 x 16
int hidden_nodes = 96;
int second_hidden_nodes = 48;

int output_nodes = 7; // t, u, v, x, y, z
double learning_rate = 0.002;
int epochs = 175;

"""

# forward propagation 구현  

1. input layer  
(1, input_nodes)a1, z1 = input data

2. first hidden layer  
w2(input_nodes, hidden_nodes) -> 내적으로 결과 shape (1, hidden_nodes)로 변경  
편향을 더해줌  
(1, hidden_nodes)z2 = a1 * w2 + b2   
(1, hidden_nodes)a2 = relu(z2)  

3. second hidden layer  
w3(hidden_nodes, second_hidden_nodes) -> 내적으로 결과 shape (1, second_hidden_nodes)로 변경  
편향을 더해줌  
(1, second_hidden_nodes)z3 = a2 * w3 + b3  
(1, second_hidden_nodes)a3 = relu(z3)  

4. output layer  
w3(second_hidden_nodes, output_nodes) -> 내적으로 결과 shape (1, output_nodes)로 변경  
편향을 더해줌  
(1, output_nodes)z4 = a3 * w4 + b4  
(1, output_nodes)a4 = softmax(z4) 

5. loss 계산   

나온 예측 a4와 정답 값 target_data를 이용해 cross entropy loss 적용  

6. loss 반환  
계산된 loss 반환  




In [ ]:
"""

double forward(double** input_data, double** target_data) {
    double delta = 1e-7; // log 무한대 발산 방지

    cross entropy loss 결과 담을 변수들 동적 할당
    double** result1 = (double**)malloc(1 * sizeof(double*));
    result1[0] = (double*)malloc(output_nodes * sizeof(double));

    double** result2 = (double**)malloc(1 * sizeof(double*));
    result2[0] = (double*)malloc(output_nodes * sizeof(double));

    double** result = (double**)malloc(1 * sizeof(double*));
    result[0] = (double*)malloc(output_nodes * sizeof(double));

    // 처음 input data를 z1, a1에 넣어줌
    for (int i = 0; i < input_nodes; i++) {
        z1[0][i] = input_data[0][i];
        a1[0][i] = input_data[0][i];
    }

    // forward propagation

    // z2 = a1 * w2 + b2
    // a2 = relu(z2) 
    dot_product(z2, a1, w2, 1, input_nodes, hidden_nodes);
    elementwise_add(z2, z2, b2, 1, hidden_nodes);
    relu(a2, z2, 1, hidden_nodes);

    // z3 = a2 * w3 + b3
    // a3 = relu(z3)
    dot_product(z3, a2, w3, 1, hidden_nodes, second_hidden_nodes);
    elementwise_add(z3, z3, b3, 1, second_hidden_nodes);
    relu(a3, z3, 1, second_hidden_nodes);

    // z4 = a3 * w4 + b4
    // a4 = softmax(z4)
    dot_product(z4, a3, w4, 1, second_hidden_nodes, output_nodes);
    elementwise_add(z4, z4, b4, 1, output_nodes);
    softmax(a4, z4, 1, output_nodes);

    // cross entropy loss 
    double** log_result = (double**)malloc(1 * sizeof(double*));
    log_result[0] = (double*)malloc(output_nodes * sizeof(double));

    // a4 + delta 발산 방지
    add_matrix(a4, 1, output_nodes, delta);
    // log(a4) = log_result
    log_matrix(log_result, a4, 1, output_nodes);
    // target_data * log(a4) = target_data * log_result = result1
    elementwise_mul(result1, target_data, log_result, 1, output_nodes);

    double** a = (double**)malloc(1 * sizeof(double*));
    a[0] = (double*)malloc(output_nodes * sizeof(double));
    double** b = (double**)malloc(1 * sizeof(double*));
    b[0] = (double*)malloc(output_nodes * sizeof(double));

    // 1 - target_data a에 넣어줌
    matrix_minus_from_one(a, target_data, 1, output_nodes);
    // 1 - a4 b에 넣어줌
    matrix_minus_from_one(b, a4, 1, output_nodes);
    // 1 - a4 + delta 발산 방지
    add_matrix(b, 1, output_nodes, delta);
    // log(1 - a4)
    log_matrix(b, b, 1, output_nodes);
    // (1 - target_data) * log(1 - a4) = a * b = result2
    elementwise_mul(result2, a, b, 1, output_nodes);
    // result = target_data * log(a4) + (1 - target_data) * log(1 - a4) = result1 + result2
    elementwise_add(result, result1, result2, 1, output_nodes);

    free(result1);
    free(result2);
    free(log_result);
    free(a);
    free(b);

    // loss 반환
    return -1 * sum(result, 1, output_nodes);
}

"""

# backward propagation 구현  

1. forward 함수 호출해 우선 forward 진행  

2. output layer 계층 역전파  
loss4 = dl/dz4를 구함  
loss4 = (a4 - target_data) = dl/da4 * da4/dz4 = dl/dz4  
z4 = w4 * a3 + b4 -> dz4/dw4 = a3  
연산을 위해 a3 transpose    

w4 업데이트  
dw4 = a3_T * loss4  = dl/dz4 * dz4/dw4 = dl/dw4  
dw4 = dl/dw4 * learning_rate  
w4 = w4 - dw4  

b4 업데이트  
db4 = dl/dz4 * dz4/db4 = dl/db4, dl/db4 = 1  
db4 = dl/db4 * learning_rate 
b4 = b4 - db4  

3. second hidden layer 계층 역전파  
loss3 = dl/dz3를 구함  
z4 = w4 * a3 + b4 -> dz4/da3 = w4  
계산을 위해 w4 transpose  
loss3 = dl/dz4 * dz4/da3 = dl/da3 * da3/dz3= loss4 * w4_T * relu_back(a3)  

z3 = w3 * a2 + b3 -> dz3/dw3 = a2  
연산을 위해 a2 transpose   

w3 업데이트  
dw3 = a2_T * loss3  = dl/dz3 * dz3/dw3 = dl/dw3  
dw3 = dl/dw3 * learning_rate  
w3 = w3 - dw3  

b3 업데이트  
db3 = dl/dz3 * dz3/db3 = dl/db3, dl/db3 = 1  
db3 = dl/db3 * learning_rate 
b3 = b3 - db3  

4. hidden layer 계층 역전파  
loss2 = dl/dz2를 구함  
z3 = w3 * a2 + b3 -> dz3/da2 = w3  
계산을 위해 w3 transpose  
loss2 = dl/dz3 * dz3/da2 = dl/da2 * da2/dz2 = loss3 * w3_T * relu_back(a2)  

z2 = w2 * a1 + b2 -> dz2/dw2 = a1  
연산을 위해 a1 transpose    

w2 업데이트  
dw2 = a2_T * loss2  = dl/dz2 * dz2/dw2 = dl/dw2  
dw2 = dl/dw2 * learning_rate  
w2 = w2 - dw2  

b2 업데이트  
db2 = dl/dz2 * dz2/db2 = dl/db2, dl/db2 = 1  
db2 = dl/db2 * learning_rate 
b2 = b2 - db2  

In [ ]:
"""
double train(double** input_data, double** target_data) {

    // forward propagation
    double loss_result = forward(input_data, target_data);

    double** loss4 = (double**)malloc(1 * sizeof(double*));
    loss4[0] = (double*)malloc(output_nodes * sizeof(double));
    double** loss3 = (double**)malloc(1 * sizeof(double*));
    loss3[0] = (double*)malloc(second_hidden_nodes * sizeof(double));
    double** loss2 = (double**)malloc(1 * sizeof(double*));
    loss2[0] = (double*)malloc(hidden_nodes * sizeof(double));

    // back propagation
    //  loss4 = (a4 - target_data) = dl/da4 * da4/dz4 = dl/dz4
    elementwise_minus(loss4, a4, target_data, 1, output_nodes);

    // 미리 연산 결과를 담을 변수들 동적 할당
    double** a_T = (double**)malloc(second_hidden_nodes * sizeof(double*));
    for (int i = 0; i < second_hidden_nodes; i++) a_T[i] = (double*)malloc(1 * sizeof(double));

    double** dw = (double**)malloc(second_hidden_nodes * sizeof(double*));
    for (int i = 0; i < second_hidden_nodes; i++) dw[i] = (double*)malloc(output_nodes * sizeof(double));

    double** db = (double**)malloc(1 * sizeof(double*));
    db[0] = (double*)malloc(output_nodes * sizeof(double));

    // z4 = w4 * a3 + b4 -> dz4/dw4 = a3 
    // 연산을 위해 a3 transpose
    matrix_transpose(a_T, a3, 1, second_hidden_nodes);
    // dw4 = a3_T * loss4 = dl/dz4 * dz4/dw4 = dl/dw4
    dot_product(dw, a_T, loss4, second_hidden_nodes, 1, output_nodes);
    // dw4 = dl/dw4 * learning_rate 
    mul_matrix(dw, dw, second_hidden_nodes, output_nodes, learning_rate);
    // w4 = w4 - dw4
    elementwise_minus(w4, w4, dw, second_hidden_nodes, output_nodes);

    // db4 = dl/dz4 * dz4/db4 = dl/db4, dl/db4 = 1
    // db4 = dl/db4 * learning_rate
    mul_matrix(db, loss4, 1, output_nodes, learning_rate);
    // b4 = b4 - db4
    elementwise_minus(b4, b4, db, 1, output_nodes);

    free(a_T);
    free(dw);
    free(db);

    double** w_T = (double**)malloc(output_nodes * sizeof(double*));
    for (int i = 0; i < output_nodes; i++) w_T[i] = (double*)malloc(second_hidden_nodes * sizeof(double));

    double** da = (double**)malloc(1 * sizeof(double*));
    da[0] = (double*)malloc(second_hidden_nodes * sizeof(double));

    double** dz = (double**)malloc(1 * sizeof(double*));
    dz[0] = (double*)malloc(second_hidden_nodes * sizeof(double));

    // z4 = w4 * a3 + b4 -> dz4/da3 = w4
    // 계산을 위해 w4 transpose
    matrix_transpose(w_T, w4, second_hidden_nodes, output_nodes);
    // dl/dz4 * dz4/da3 = dl/da3 = loss4 * w4_T
    dot_product(da, loss4, w_T, 1, output_nodes, second_hidden_nodes);

    // relu_back = da3/dz3 
    relu_back(dz, a3, 1, second_hidden_nodes);

    // loss3 = dl/da3 * da3/dz3 = dl/dz3 = loss4 * w4_T * relu_back(a3)
    elementwise_mul(loss3, da, dz, 1, second_hidden_nodes);


    a_T = (double**)malloc(hidden_nodes * sizeof(double*));
    for (int i = 0; i < hidden_nodes; i++) a_T[i] = (double*)malloc(1 * sizeof(double));

    dw = (double**)malloc(hidden_nodes * sizeof(double*));
    for (int i = 0; i < hidden_nodes; i++) dw[i] = (double*)malloc(second_hidden_nodes * sizeof(double));

    db = (double**)malloc(1 * sizeof(double*));
    db[0] = (double*)malloc(second_hidden_nodes * sizeof(double));

    // z3 = w3 * a2 + b3 -> dz3/dw3 = a2
    // 계산을 위해 a2 transpose
    matrix_transpose(a_T, a2, 1, hidden_nodes);
    // dw3 = a2_T * loss3 = dl/dz3 * dz3/dw3 = dl/dw3
    dot_product(dw, a_T, loss3, hidden_nodes, 1, second_hidden_nodes);
    // dw3 = dl/dw3 * learning_rate
    mul_matrix(dw, dw, hidden_nodes, second_hidden_nodes, learning_rate);
    // w3 = w3 - dw3
    elementwise_minus(w3, w3, dw, hidden_nodes, second_hidden_nodes);

    // db3 = dl/dz3 * dz3/db3 = dl/db3, dl/db3 = 1
    // db3 = dl/db3 * learning_rate
    mul_matrix(db, loss3, 1, second_hidden_nodes, learning_rate);
    // b3 = b3 - db3
    elementwise_minus(b3, b3, db, 1, second_hidden_nodes);

    free(w_T);
    free(da);
    free(dz);
    free(a_T);
    free(dw);
    free(db);

    w_T = (double**)malloc(second_hidden_nodes * sizeof(double*));
    for (int i = 0; i < second_hidden_nodes; i++) w_T[i] = (double*)malloc(hidden_nodes * sizeof(double));

    da = (double**)malloc(1 * sizeof(double*));
    da[0] = (double*)malloc(hidden_nodes * sizeof(double));

    dz = (double**)malloc(1 * sizeof(double*));
    dz[0] = (double*)malloc(hidden_nodes * sizeof(double));

    // z3 = w3 * a2 + b3 -> dz3/da2 = w3
    // 계산을 위해 w3 transpose
    matrix_transpose(w_T, w3, hidden_nodes, second_hidden_nodes);
    // dl/dz3 * dz3/da2 = dl/da2 = = loss3 * w3_T
    dot_product(da, loss3, w_T, 1, second_hidden_nodes, hidden_nodes);
    // relu_back = da2/dz2
    relu_back(dz, a2, 1, hidden_nodes);
    // loss2 = dl/da2 * da2/dz2 = dl/dz2 = loss3 * w3_T * relu_back(a2)
    elementwise_mul(loss2, da, dz, 1, hidden_nodes);

    a_T = (double**)malloc(input_nodes * sizeof(double*));
    for (int i = 0; i < input_nodes; i++) a_T[i] = (double*)malloc(1 * sizeof(double));

    dw = (double**)malloc(input_nodes * sizeof(double*));
    for (int i = 0; i < input_nodes; i++) dw[i] = (double*)malloc(hidden_nodes * sizeof(double));

    db = (double**)malloc(1 * sizeof(double*));
    db[0] = (double*)malloc(hidden_nodes * sizeof(double));

    // z2 = w2 * a1 + b2 -> dz2/dw2 = a1
    // 계산을 위해 a1 transpose
    matrix_transpose(a_T, a1, 1, input_nodes);
    // dw2 = a2_T * loss2 = dl/dz2 * dz2/dw2 = dl/dw2
    dot_product(dw, a_T, loss2, input_nodes, 1, hidden_nodes);
    // dw2 = dl/dw2 * learning_rate
    mul_matrix(dw, dw, input_nodes, hidden_nodes, learning_rate);
    // w2 = w2 - dw2
    elementwise_minus(w2, w2, dw, input_nodes, hidden_nodes);

    // db2 = dl/dz2 * dz2/db2 = dl/db2, dl/db2 = 1
    // db2 = dl/db2 * learning_rate
    mul_matrix(db, loss2, 1, hidden_nodes, learning_rate);
    // b2 = b2 - db2
    elementwise_minus(b2, b2, db, 1, hidden_nodes);

    free(w_T);
    free(da);
    free(dz);
    free(a_T);
    free(dw);
    free(db);

    free(loss2);
    free(loss3);
    free(loss4);

    return loss_result;
}

"""

# Predict 함수  

테스트 데이터를 하나 받아와 학습된 가중치로 forward 진행한 뒤  
output layer 결과 값 중 가장 큰 값의 index를 반환함  
가장 큰 값은 one hot encoding에서 예측한 정답을 의미함  
{0:t, 1:u, 2:v, 3:w, 4:x, 5:y, 6:z}

In [ ]:
"""
int predict(double** input_data) {

    double** Z2;
    double** A2;
    double** Z3;
    double** A3;
    double** Z4;
    double** A4;

    Z2 = (double**)malloc(1 * sizeof(double*));
    Z2[0] = (double*)malloc(hidden_nodes * sizeof(double));
    zeros(Z2, 1, hidden_nodes);

    A2 = (double**)malloc(1 * sizeof(double*));
    A2[0] = (double*)malloc(hidden_nodes * sizeof(double));
    zeros(A2, 1, hidden_nodes);

    Z3 = (double**)malloc(1 * sizeof(double*));
    Z3[0] = (double*)malloc(second_hidden_nodes * sizeof(double));
    zeros(Z3, 1, second_hidden_nodes);

    A3 = (double**)malloc(1 * sizeof(double*));
    A3[0] = (double*)malloc(second_hidden_nodes * sizeof(double));
    zeros(A3, 1, second_hidden_nodes);

    Z4 = (double**)malloc(1 * sizeof(double*));
    Z4[0] = (double*)malloc(output_nodes * sizeof(double));
    zeros(Z4, 1, output_nodes);

    A4 = (double**)malloc(1 * sizeof(double*));
    A4[0] = (double*)malloc(output_nodes * sizeof(double));
    zeros(A4, 1, output_nodes);

    // 기본적인 구조 forward와 동일 
    dot_product(Z2, input_data, w2, 1, input_nodes, hidden_nodes);
    elementwise_add(Z2, Z2, b2, 1, hidden_nodes);
    relu(A2, Z2, 1, hidden_nodes);

    dot_product(Z3, A2, w3, 1, hidden_nodes, second_hidden_nodes);
    elementwise_add(Z3, Z3, b3, 1, second_hidden_nodes);
    relu(A3, Z3, 1, second_hidden_nodes);

    dot_product(Z4, A3, w4, 1, second_hidden_nodes, output_nodes);
    elementwise_add(Z4, Z4, b4, 1, output_nodes);
    softmax(A4, Z4, 1, output_nodes);

    // 출력값 중 가장 큰 값의 인덱스를 리턴
    int predicted_num = find_max_index(A4, output_nodes);

    free(Z2);
    free(A2);
    free(Z3);
    free(A3);
    free(Z4);
    free(A4);

    return predicted_num;
}




"""

# accuracy 함수  
받아온 test data를 0.01 ~ 1.0의 값을 가지게끔 정규화를 한 뒤에  
predict 함수를 통해 예측을 진행함  
정답 라벨링 값과 예측한 값이 같으면 cnt를 1 더 해주고  
모든 test data에 대해 같은 과정을 진행한 뒤 총 정확도를 출력함

In [ ]:
"""
void accuracy(double input_data[140][257]) {

    double count = 0;
    double** data = (double**)malloc(1 * sizeof(double*));
    data[0] = (double*)malloc(input_nodes * sizeof(double));

    // test data 140개를 predict 함수를 통해 예측한 값과 label이 같으면 count를 증가시킴
    for (int i = 0; i < 140; i++) {
        int label = input_data[i][256];
        for (int j = 0; j < 256; j++) {
            data[0][j] = (input_data[i][j] / 255.0 * 0.99) + 0.01;
        }
        int predicted_num = predict(data);

        if (label == predicted_num) count++;
    }
    printf("count %lf, Current Accuracy = %lf\n", count, 100 * (count / 140));
    free(data);
}

"""

# init 함수  
계산에 사용할 변수들 미리 동적할당 함  

w2(input_nodes, hidden_nodes)  
w3(hidden_nodes, second_hidden_nodes)  
w4(second_hidden_nodes, output_nodes)  

b2(1, hidden_nodes)  
b3(1, second_hidden_nodes)  
b4(1, output_nodes)

z1(1,input_nodes)  
a1(1,input_nodes)  

z2(1,hidden_nodes)    
a2(1,hidden_nodes)  

z3(1,second_hidden_nodes)  
a3(1,second_hidden_nodes)  

z4(1,output_nodes)    
a4(1,output_nodes)  

가중치의 경우 he initalization을 통해 초기화를 함  

정규 분포의 난수로 이루어진 것을 입력에 제곱근을 씌워준 것으로 나누는 xavier inialization에 2를 곱해준 형태임  

편향은 0~1 난수로 만듬  

In [ ]:
"""

void init()
{
    // input layer, hidden layer, output layer 선형회귀 값, 출력 값 선언 및 가중치 설정 
    w2 = (double**)malloc(input_nodes * sizeof(double*));
    for (int i = 0; i < input_nodes; i++) w2[i] = (double*)malloc(hidden_nodes * sizeof(double));
    randnArray(w2, input_nodes, hidden_nodes);
    divide_matrix(w2, input_nodes, hidden_nodes, sqrt(input_nodes / 2)); // he 초기화

    b2 = (double**)malloc(1 * sizeof(double*));
    b2[0] = (double*)malloc(hidden_nodes * sizeof(double));
    randArray(b2, 1, hidden_nodes);

    w3 = (double**)malloc(hidden_nodes * sizeof(double*));
    for (int i = 0; i < hidden_nodes; i++) w3[i] = (double*)malloc(second_hidden_nodes * sizeof(double));
    randnArray(w3, hidden_nodes, second_hidden_nodes);
    divide_matrix(w3, hidden_nodes, second_hidden_nodes, sqrt(hidden_nodes / 2));

    b3 = (double**)malloc(1 * sizeof(double*));
    b3[0] = (double*)malloc(second_hidden_nodes * sizeof(double));
    randArray(b3, 1, second_hidden_nodes);

    w4 = (double**)malloc(second_hidden_nodes * sizeof(double*));
    for (int i = 0; i < second_hidden_nodes; i++) w4[i] = (double*)malloc(output_nodes * sizeof(double));
    randnArray(w4, second_hidden_nodes, output_nodes);
    divide_matrix(w4, second_hidden_nodes, output_nodes, sqrt(second_hidden_nodes / 2));

    b4 = (double**)malloc(1 * sizeof(double*));
    b4[0] = (double*)malloc(output_nodes * sizeof(double));
    randArray(b4, 1, output_nodes);

    // z, a 동적 할당 및 초기화
    z1 = (double**)malloc(1 * sizeof(double*));
    z1[0] = (double*)malloc(input_nodes * sizeof(double));
    zeros(z1, 1, input_nodes);

    a1 = (double**)malloc(1 * sizeof(double*));
    a1[0] = (double*)malloc(input_nodes * sizeof(double));
    zeros(a1, 1, input_nodes);

    z2 = (double**)malloc(1 * sizeof(double*));
    z2[0] = (double*)malloc(hidden_nodes * sizeof(double));
    zeros(z2, 1, hidden_nodes);

    a2 = (double**)malloc(1 * sizeof(double*));
    a2[0] = (double*)malloc(hidden_nodes * sizeof(double));
    zeros(a2, 1, hidden_nodes);

    z3 = (double**)malloc(1 * sizeof(double*));
    z3[0] = (double*)malloc(second_hidden_nodes * sizeof(double));
    zeros(z3, 1, second_hidden_nodes);

    a3 = (double**)malloc(1 * sizeof(double*));
    a3[0] = (double*)malloc(second_hidden_nodes * sizeof(double));
    zeros(a3, 1, second_hidden_nodes);

    z4 = (double**)malloc(1 * sizeof(double*));
    z4[0] = (double*)malloc(output_nodes * sizeof(double));
    zeros(z4, 1, output_nodes);

    a4 = (double**)malloc(1 * sizeof(double*));
    a4[0] = (double*)malloc(output_nodes * sizeof(double));
    zeros(a4, 1, output_nodes);
}

"""

# main 함수  

train data 파일 315개 test data 파일 140개를 받아옴  
라벨을 기준으로 class마다 one hot encoding을 해준 뒤  
정규화된 학습 데이터로 epochs만큼 학습을 함  
50번마다 loss를 출력하고 학습이 끝날 때마다 정확도를 출력함  
마지막에 hyper parameter들을 출력함 


In [ ]:
"""

int main() {

    // 시드설정 가중치를 위한
    srand(time(NULL));

    // test파일 train 파일 불러오기
    FILE* file;
    char buffer[1024];
    int i, j;
    double test_data[140][257];
    double train_data[315][257];

    file = fopen("test.csv", "r");
    i = 0;
    while (fgets(buffer, sizeof(buffer), file) != NULL) {
        char* token = strtok(buffer, ",");
        j = 0;
        while (token != NULL) {
            test_data[i][j] = atoi(token);
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    fclose(file);

    file = fopen("train.csv", "r");
    i = 0;
    while (fgets(buffer, sizeof(buffer), file) != NULL) {
        char* token = strtok(buffer, ",");
        j = 0;
        while (token != NULL) {
            train_data[i][j] = atoi(token);
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    // 파일 닫기
    fclose(file);

    init();

    for (int i = 0; i < epochs; i++) {

        for (int step = 0; step < 315; step++) {

            // target data, input data 동적 할당 및 초기화
            double** target_data = (double**)malloc(1 * sizeof(double*));
            target_data[0] = (double*)malloc(output_nodes * sizeof(double));
            zeros(target_data, 1, output_nodes);
            // 0.01로 초기화
            add_matrix(target_data, 1, output_nodes, 0.01);

            double** input_data = (double**)malloc(1 * sizeof(double*));
            input_data[0] = (double*)malloc(input_nodes * sizeof(double));
            int label = train_data[step][256];
            // one hot encoding 
            // 정답 레이블에 해당하는 인덱스에 0.99를 넣어줌 나머지는 0.01
            target_data[0][label] = 0.99;

            for (int j = 0; j < 256; j++) {
                // 이미지 데이터 0~255 -> 0.01~1.0 
                // 정규화
                input_data[0][j] = (train_data[step][j] / 255.0 * 0.99) + 0.01;
            }
            double loss_result = train(input_data, target_data);

            // 계산 도중에 z2의 값이 무한대로 갔음 0.00001으로 나눈 것을 0으로 인식해버림 가중치 재설정하고 학습 처음부터 다시 시작함.
            // 초기 한번만 CHECK하면 이후에 무한대로 가는 경우는 없었음. 따라서 이런 식으로 하드코딩
            for (int i = 0; i < hidden_nodes; i++) {
                if (isinf(z2[0][i])) {
                    free(w2);
                    free(b2);
                    free(w3);
                    free(b3);
                    free(w4);
                    free(b4);
                    free(a1);
                    free(z1);
                    free(a2);
                    free(z2);
                    free(a3);
                    free(z3);
                    free(a4);
                    free(z4);
                    init();
                    i = 0;
                    step = -1;
                }
            }
            // 50번마다 loss 출력
            if (step % 50 == 0) {
                printf("current epochs = %d step = %d, loss_val = %lf\n", i, step, loss_result);
            }

            free(target_data);
            free(input_data);
        }
        // 한번 학습이 끝날 때마다 정확도 출력
        accuracy(test_data);
    }

    free(w2);
    free(b2);
    free(w3);
    free(b3);
    free(w4);
    free(b4);
    free(a1);
    free(z1);
    free(a2);
    free(z2);
    free(a3);
    free(z3);
    free(a4);
    free(z4);

    
    printf("input_nodes: %d\n", input_nodes);
    printf("hidden_nodes: %d\n", hidden_nodes);
    printf("second_hidden_nodes: %d\n", second_hidden_nodes);
    printf("output_nodes: %d\n", output_nodes);
    printf("learning_rate: %lf\n", learning_rate);
    printf("epochs: %d\n", epochs);
    
    return 0;
}

"""

![nn](c_result/result1.JPG)

초기 설정한 파라미터로 학습한 결과  
오버피팅이 된 모습을 확인할 수 있다.  

![nn](c_result/result2.JPG)  


학습횟수를 어떻게 조절하느냐에 따라 결과가 완전히 다른 모습을 확인할 수 있었다.  
이 파라미터에서 학습률만 올려서 다시 실행해 보았다.  


![nn](c_result/result3.JPG)  

이전 결과보다 훨씬 빠르게 오버피팅이 된 것을 볼 수 있었다.  

반대로 학습률을 낮춘다음에 다시 학습을 시켜봤다.  


![nn](c_result/result4.JPG)  

best accruacy는 아니지만 오버피팅을 방지할 순 있었다.  
적절한 학습률과 학습횟수를 미리 설정해서 돌린다면  
최선의 결과를 얻을 수 있을 것이다. 


초기 하이퍼 파라미터에서  레이어의 수를 줄이고 학습을 진행해보았다.  

![nn](c_result/result5.JPG)  

오버피팅이 되는 지점을 찾기 위해 학습 횟수를 늘려보았다.  

67번째 학습부터 정확도가 감소하기 시작하였고 75에 이르러선 완전히 바닥에 도달한 것을 볼 수 있었다.    


![nn](c_result/result6.JPG)

실험의 결과 정상적으로 학습됐을 경우 랜덤 생성된 가중치에 따라 정확도가 91 ~ 95선에서 모델이 완성되는 것을 확인할 수 있었고  
오버피팅 지점을 넘어서는 순간 loss값이 매우 커지고 정확도가 바닥에 이르는 것을 확인할 수 있었다.  

적절한 하이퍼 파라미터를 찾아 좋게 학습된 결과 

-![nn](c_result/result_7.JPG)  


매번 저렇게 좋은 결과가 나오진 않지만 최소한 최악의 경우인 정확도가 14 정도가 나오는  
오버피팅 지점에 빠지진 않게끔 하이퍼 파라미터를 조정할 수는 있었다.  

학습을 빠르게 진행하고 싶다면 학습률을 높이고 학습횟수는 줄이면 되고  
반대의 경우도 가능하다. 학습률을 낮추고 학습횟수를 늘리면 천천히 학습을 진행할 수 있다.  

레이어의 수 같은 경우   
레이어의 수를 작게 하면 학습이 느려 학습횟수를 많이 가져가야 한다는 단점이 있었다.  

-![nn](c_result/result9.JPG)  


다음과 같이 학습을 100번 했을 때 결과가 좋게 나오는 것을 확인할 수 있었다.    
 

레이어의 수를 많이 키우면 학습 횟수가 많이 필요하진 않지만  
한 번의 학습을 할 때 시간이 오래 걸리고 또 파이썬 같은 경우에는 c에 비해 속도가 느려 더 느리게 된다.  

레이어의 수를 늘리고 싶을 떄 적절한 하이퍼 파라미터는  
first hidden layer는 60 ~ 80  
second hidden layer는 30 ~ 40  
학습률은 0.001  
학습횟수는 40 ~ 50이고  

레이어의 수를 낮추고 싶을 때 적절한 하이퍼 파라미터는  
레이어를 30 ~ 40 그리고 15 ~ 20으로 내리고  
학습률을 올리거나 학습 횟수를 낮추는 것이다.  

두 방식 다 중요한 것은 오버피팅이 되는 지점을 찾고 그 지점까지 가지 않게끔 본인이 원하는 하이퍼파라미터로 조정하는 것이다.  

레이어의 수를 줄이고 학습률이나 학습횟수를 높이거나  
레이어의 수를 높이고 학습률이나 학습횟수를 줄이는 것은  
본인의 취향에 달린 것 같다.  

오버피팅에 빠지지 않는다면 정확도는 초기 생성되는 가중치 그리고 학습되는 정도에 따라 90 ~ 97에 학습이 완료되는 것을 확인할 수 있었다.